1)Write a python program which searches all the product under a particular product from www.amazon.in. The
product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for
guitars.

In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install pandas
!pip install geopy

In [ ]:
import requests
from bs4 import BeautifulSoup

def search_amazon_products(search_query):
    base_url = f'https://www.amazon.in/s?k={search_query.replace(" ", "+")}'
    headers = {
        'User-Agent': 'Your User Agent String',
    }

    try:
        response = requests.get(base_url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find and print product titles
        product_titles = soup.find_all('span', {'class': 'a-size-medium a-color-base a-text-normal'})
        for index, title in enumerate(product_titles, start=1):
            print(f'{index}. {title.text.strip()}')

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

if __name__ == '__main__':
    search_query = input("Enter the product you want to search on Amazon.in: ")
    search_amazon_products(search_query)

2)In the above question, now scrape the following details of each product listed in first 3 pages of your search
results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then
scrape all the products available under that product name. Details to be scraped are: "Brand
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“. 

In [ ]:
# Function to scrape Amazon product details
def scrape_amazon_products(search_query, max_pages=3):
    base_url = f'https://www.amazon.in/s?k={search_query.replace(" ", "+")}'
    headers = {
        'User-Agent': 'Your User Agent String',
    }

    product_data = {
        "Brand Name": [],
        "Name of the Product": [],
        "Price": [],
        "Return/Exchange": [],
        "Expected Delivery": [],
        "Availability": [],
        "Product URL": [],
    }

    page_num = 1
    while page_num <= max_pages:
        url = f'{base_url}&page={page_num}'
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        products = soup.find_all('div', {'data-asin': True})
        if not products:
            break

        for product in products:
            product_url = f"https://www.amazon.in{product.find('a', {'class': 'a-link-normal'})['href']}"
            brand_name = product.find('span', {'class': 'a-size-base-plus'}).text.strip()
            product_name = product.find('span', {'class': 'a-text-normal'}).text.strip()
            price = product.find('span', {'class': 'a-price'}).text.strip()
            return_exchange = product.find('div', {'class': 'a-row a-size-base a-color-secondary'}).text.strip()
            expected_delivery = product.find('div', {'class': 'a-row s-align-children-center'}).text.strip()
            availability = product.find('div', {'class': 'a-row a-size-base a-color-secondary'}).find_next('div').text.strip()

            product_data["Brand Name"].append(brand_name)
            product_data["Name of the Product"].append(product_name)
            product_data["Price"].append(price)
            product_data["Return/Exchange"].append(return_exchange)
            product_data["Expected Delivery"].append(expected_delivery)
            product_data["Availability"].append(availability)
            product_data["Product URL"].append(product_url)

        page_num += 1

    return pd.DataFrame(product_data)

# Input from the user
search_query = input("Enter the product you want to search on Amazon: ")
max_pages = int(input("Enter the maximum number of pages to scrape: "))

# Scrape and save data to a CSV
data_frame = scrape_amazon_products(search_query, max_pages)
data_frame.to_csv(f'{search_query}_products.csv', index=False)

3)Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’. 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time
import requests

# Function to scrape images from Google Images
def scrape_google_images(search_keywords, num_images=10):
    driver = webdriver.Chrome(executable_path="path/to/chromedriver")
    driver.get("https://images.google.com/")
    driver.maximize_window()

    image_urls = []

    for keyword in search_keywords:
        search_bar = driver.find_element(By.NAME, "q")
        search_bar.clear()
        search_bar.send_keys(keyword)
        search_bar.send_keys(Keys.RETURN)

        time.sleep(2)

        # Scroll down to load more images
        for _ in range(3):  # Scroll down 3 times
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

        # Get image URLs
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        img_tags = soup.find_all('img', class_='rg_i')

        for img in img_tags[:num_images]:
            if img.get('data-src'):
                image_urls.append(img.get('data-src'))

    driver.quit()
    return image_urls

# Function to download and save images
def download_and_save_images(image_urls, save_path='./images/'):
    for i, url in enumerate(image_urls):
        response = requests.get(url)
        if response.status_code == 200:
            with open(f'{save_path}image_{i+1}.jpg', 'wb') as file:
                file.write(response.content)

if __name__ == "__main__":
    search_keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']
    num_images_per_keyword = 10

    image_urls = scrape_google_images(search_keywords, num_images_per_keyword)
    download_and_save_images(image_urls)

4)Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com
and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand
Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the
details is missing then replace it by “- “. Save your results in a dataframe and CSV. 

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to scrape smartphone details from Flipkart
def scrape_flipkart_smartphones(search_query):
    base_url = f'https://www.flipkart.com/search?q={search_query.replace(" ", "+")}'
    headers = {
        'User-Agent': 'Your User Agent String',
    }

    product_data = {
        "Brand Name": [],
        "Smartphone Name": [],
        "Colour": [],
        "RAM": [],
        "Storage(ROM)": [],
        "Primary Camera": [],
        "Secondary Camera": [],
        "Display Size": [],
        "Battery Capacity": [],
        "Price": [],
        "Product URL": [],
    }

    try:
        response = requests.get(base_url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        products = soup.find_all('div', {'class': '_1AtVbE'})
        for product in products:
            brand_name = product.find('div', {'class': '_4rR01T'}).text.strip()
            smartphone_name = product.find('a', {'class': 'IRpwTa'}).text.strip()
            color = product.find('div', {'class': '_1I3t12'}).text.strip()
            specs = product.find_all('li', {'class': 'rgWa7D'})
            ram, storage, primary_camera, secondary_camera, display_size, battery_capacity = ["-"] * 6
            for spec in specs:
                text = spec.text.strip()
                if "RAM" in text:
                    ram = text
                elif "ROM" in text:
                    storage = text
                elif "Primary Camera" in text:
                    primary_camera = text
                elif "Secondary Camera" in text:
                    secondary_camera = text
                elif "Display Size" in text:
                    display_size = text
                elif "Battery Capacity" in text:
                    battery_capacity = text
            price = product.find('div', {'class': '_30jeq3'}).text.strip()
            product_url = f"https://www.flipkart.com{product.find('a', {'class': 'IRpwTa'})['href']}"

            product_data["Brand Name"].append(brand_name)
            product_data["Smartphone Name"].append(smartphone_name)
            product_data["Colour"].append(color)
            product_data["RAM"].append(ram)
            product_data["Storage(ROM)"].append(storage)
            product_data["Primary Camera"].append(primary_camera)
            product_data["Secondary Camera"].append(secondary_camera)
            product_data["Display Size"].append(display_size)
            product_data["Battery Capacity"].append(battery_capacity)
            product_data["Price"].append(price)
            product_data["Product URL"].append(product_url)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

    return pd.DataFrame(product_data)

if __name__ == '__main':
    search_query = input("Enter the smartphone you want to search on Flipkart: ")
    data_frame = scrape_flipkart_smartphones(search_query)
    data_frame.to_csv(f'{search_query}_smartphones.csv', index=False)

5)Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps. 

In [ ]:
from geopy.geocoders import GoogleV3

def get_coordinates(city_name):
    api_key = 'https://developers.google.com/maps/gmp-get-started'  # Replace with your Google Maps API key
    geolocator = GoogleV3(api_key=api_key)
    
    try:
        location = geolocator.geocode(city_name)
        if location:
            latitude, longitude = location.latitude, location.longitude
            return latitude, longitude
        else:
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

if __name__ == '__main__':
    city_name = input("Enter the name of the city: ")
    coordinates = get_coordinates(city_name)
    
    if coordinates:
        print(f"Coordinates for {city_name}: Latitude {coordinates[0]}, Longitude {coordinates[1]}")
    else:
        print(f"Coordinates for {city_name} not found.")

6)Write a program to scrap all the available details of best gaming laptops from digit.in.

In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_gaming_laptops():
    url = 'https://www.digit.in/top-products/best-gaming-laptops-40.html'
    headers = {
        'User-Agent': 'Your User Agent String',
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        laptops_data = []

        product_cards = soup.find_all('div', class_='TopNumbeHeading')
        for card in product_cards:
            laptop_data = {}

            title = card.find('div', class_='TopNumbeHeading').text
            specs = card.find('div', class_='Specs').text
            price = card.find('div', class_='Price').text
            ratings = card.find('div', class_='starRating').text

            laptop_data['Title'] = title
            laptop_data['Specs'] = specs
            laptop_data['Price'] = price
            laptop_data['Ratings'] = ratings

            laptops_data.append(laptop_data)

        return laptops_data

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

if __name__ == '__main':
    gaming_laptops = scrape_gaming_laptops()

    if gaming_laptops:
        for index, laptop in enumerate(gaming_laptops, start=1):
            print(f"Laptop {index}:")
            print(f"Title: {laptop['Title']}")
            print(f"Specs: {laptop['Specs']}")
            print(f"Price: {laptop['Price']}")
            print(f"Ratings: {laptop['Ratings']}")
            print()

        # You can save the data to a CSV file if needed
        # Example: pd.DataFrame(gaming_laptops).to_csv('gaming_laptops.csv', index=False)
    else:
        print("No gaming laptops data found.")

7)Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped:
“Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”. 

In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_forbes_billionaires():
    url = 'https://www.forbes.com/billionaires/'
    headers = {
        'User-Agent': 'Your User Agent String',
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        billionaires_data = []

        rows = soup.find_all('div', class_='rank')
        for row in rows:
            billionaire_data = {}
            
            rank = row.find('span', class_='num').text.strip()
            name = row.find('h2', class_='title').text.strip()
            net_worth = row.find('div', class_='netWorth').text.strip()
            age = row.find('div', class_='age').text.strip()
            citizenship = row.find('div', class_='countryOfCitizenship').text.strip()
            source = row.find('div', class_='source').text.strip()
            industry = row.find('div', class_='category').text.strip()

            billionaire_data['Rank'] = rank
            billionaire_data['Name'] = name
            billionaire_data['Net worth'] = net_worth
            billionaire_data['Age'] = age
            billionaire_data['Citizenship'] = citizenship
            billionaire_data['Source'] = source
            billionaire_data['Industry'] = industry

            billionaires_data.append(billionaire_data)

        return billionaires_data

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

if __name__ == '__main':
    billionaires = scrape_forbes_billionaires()

    if billionaires:
        for billionaire in billionaires:
            print("Rank:", billionaire['Rank'])
            print("Name:", billionaire['Name'])
            print("Net worth:", billionaire['Net worth'])
            print("Age:", billionaire['Age'])
            print("Citizenship:", billionaire['Citizenship'])
            print("Source:", billionaire['Source'])
            print("Industry:", billionaire['Industry'])
            print()

        # You can save the data to a CSV file if needed
        # Example: pd.DataFrame(billionaires).to_csv('forbes_billionaires.csv', index=False)
    else:
        print("No billionaire data found.")

8)Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted
from any YouTube Video.

In [2]:
'google-api-python-client'
!pip install google-api-python-client

In [5]:
import os
import googleapiclient.discovery

# Set your API key here
API_KEY = 'YOUR_YOUTUBE_API_KEY'

# YouTube video URL for the video you want to extract comments from
VIDEO_URL = 'https://www.youtube.com/watch?v=VIDEO_ID'

# Function to extract comments, comment upvotes, and time from a YouTube video
def extract_youtube_comments(api_key, video_url):
    video_id = video_url.split("v=")[1]
    youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=api_key)

    comments = []
    next_page_token = None

    while True:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            textFormat="plainText",
            maxResults=100,
            pageToken=next_page_token
        )
        response = request.execute()

        for comment in response["items"]:
            comment_text = comment["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
            comment_upvotes = comment["snippet"]["topLevelComment"]["snippet"]["likeCount"]
            comment_time = comment["snippet"]["topLevelComment"]["snippet"]["publishedAt"]
            comments.append({
                "Comment": comment_text,
                "Upvotes": comment_upvotes,
                "Time": comment_time
            })

        if "nextPageToken" in response:
            next_page_token = response["nextPageToken"]
        else:
            break

    return comments

if __name__ == '__main__':
    comments = extract_youtube_comments(API_KEY, VIDEO_URL)

    if comments:
        for index, comment in enumerate(comments[:500], start=1):
            print(f"Comment {index}:")
            print(f"Comment: {comment['Comment']}")
            print(f"Upvotes: {comment['Upvotes']}")
            print(f"Time: {comment['Time']}")
            print()

    else:
        print("No comments found.")

HttpError: <HttpError 400 when requesting https://youtube.googleapis.com/youtube/v3/commentThreads?part=snippet&videoId=VIDEO_ID&textFormat=plainText&maxResults=100&key=YOUR_YOUTUBE_API_KEY&alt=json returned "API key not valid. Please pass a valid API key.". Details: "[{'message': 'API key not valid. Please pass a valid API key.', 'domain': 'global', 'reason': 'badRequest'}]">

9)Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall
reviews, privates from price, dorms from price, facilities and property description. 


In [6]:
import requests
from bs4 import BeautifulSoup

def scrape_hostelworld_data():
    url = 'https://www.hostelworld.com/s?q=London&country=England'
    headers = {
        'User-Agent': 'Your User Agent String',
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        hostels_data = []

        hostel_cards = soup.find_all('div', class_='property-card')
        for card in hostel_cards:
            hostel_data = {}
            
            hostel_name = card.find('h2', class_='title-2').text.strip()
            distance_from_city_center = card.find('span', class_='reviews').find('span', {'class': 'label'}).text.strip()
            ratings = card.find('div', class_='score orange').text.strip()
            total_reviews = card.find('span', {'class': 'reviews'}).text.strip()
            overall_reviews = card.find('div', {'class': 'keyword'}).text.strip()
            privates_from_price = card.find('span', class_='price').text.strip()
            dorms_from_price = card.find('div', {'class': 'rate'}).find('div').text.strip()
            facilities = ', '.join([fac.text.strip() for fac in card.find_all('li', {'class': 'facility-badge'})])
            property_description = card.find('p', {'class': 'description'}).text.strip()

            hostel_data['Hostel Name'] = hostel_name
            hostel_data['Distance from City Center'] = distance_from_city_center
            hostel_data['Ratings'] = ratings
            hostel_data['Total Reviews'] = total_reviews
            hostel_data['Overall Reviews'] = overall_reviews
            hostel_data['Privates from Price'] = privates_from_price
            hostel_data['Dorms from Price'] = dorms_from_price
            hostel_data['Facilities'] = facilities
            hostel_data['Property Description'] = property_description

            hostels_data.append(hostel_data)

        return hostels_data

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

if __name__ == '__main':
    hostels = scrape_hostelworld_data()

    if hostels:
        for index, hostel in enumerate(hostels, start=1):
            print(f"Hostel {index}:")
            print(f"Hostel Name: {hostel['Hostel Name']}")
            print(f"Distance from City Center: {hostel['Distance from City Center']}")
            print(f"Ratings: {hostel['Ratings']}")
            print(f"Total Reviews: {hostel['Total Reviews']}")
            print(f"Overall Reviews: {hostel['Overall Reviews']}")
            print(f"Privates from Price: {hostel['Privates from Price']}")
            print(f"Dorms from Price: {hostel['Dorms from Price']}")
            print(f"Facilities: {hostel['Facilities']}")
            print(f"Property Description: {hostel['Property Description']}")
            print()

        # You can save the data to a CSV file if needed
        # Example: pd.DataFrame(hostels).to_csv('hostelworld_data.csv', index=False)
    else:
        print("No hostel data found.")